In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import preprocessing


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

sample_submission.csv
test.csv
train.csv



In [2]:
df=pd.read_csv("../input/train.csv")
df_test=pd.read_csv("../input/test.csv")

In [3]:
df.describe()

,id,bone_length,rotting_flesh,hair_length,has_soul
count,371.000000,371.000000,371.000000,371.000000,371.000000
mean,443.676550,0.434160,0.506848,0.529114,0.471392
std,263.222489,0.132833,0.146358,0.169902,0.176129
min,0.000000,0.061032,0.095687,0.134600,0.009402
25%,205.500000,0.340006,0.414812,0.407428,0.348002
50%,458.000000,0.434891,0.501552,0.538642,0.466372
75%,678.500000,0.517223,0.603977,0.647244,0.600610
max,897.000000,0.817001,0.932466,1.000000,0.935721


In [4]:
df.head()

,id,bone_length,rotting_flesh,hair_length,has_soul,color,type
0,0,0.354512,0.350839,0.465761,0.781142,clear,Ghoul
1,1,0.575560,0.425868,0.531401,0.439899,green,Goblin
2,2,0.467875,0.354330,0.811616,0.791225,black,Ghoul
3,4,0.776652,0.508723,0.636766,0.884464,black,Ghoul
4,5,0.566117,0.875862,0.418594,0.636438,green,Ghost


In [5]:
df['bone_flesh']=df['bone_length']*df['rotting_flesh']
df['bone_hair']=df['bone_length']*df['hair_length']
df['bone_soul']=df['bone_length']*df['has_soul']
df['flesh_hair']=df['rotting_flesh']*df['hair_length']
df['flesh_soul']=df['rotting_flesh']*df['has_soul']
df['hair_soul']=df['hair_length']*df['has_soul']


df.head(1)

,id,bone_length,rotting_flesh,hair_length,has_soul,color,type,bone_flesh,bone_hair,bone_soul,flesh_hair,flesh_soul,hair_soul
0,0,0.354512,0.350839,0.465761,0.781142,clear,Ghoul,0.124377,0.165118,0.276924,0.163407,0.274055,0.363825


In [6]:
df_test['bone_flesh']=df_test['bone_length']*df_test['rotting_flesh']
df_test['bone_hair']=df_test['bone_length']*df_test['hair_length']
df_test['bone_soul']=df_test['bone_length']*df_test['has_soul']
df_test['flesh_hair']=df_test['rotting_flesh']*df_test['hair_length']
df_test['flesh_soul']=df_test['rotting_flesh']*df_test['has_soul']
df_test['hair_soul']=df_test['hair_length']*df_test['has_soul']

df_test.head(1)

,id,bone_length,rotting_flesh,hair_length,has_soul,color,bone_flesh,bone_hair,bone_soul,flesh_hair,flesh_soul,hair_soul
0,3,0.471774,0.387937,0.706087,0.698537,black,0.183019,0.333114,0.329552,0.273917,0.270988,0.493228


In [7]:
dummy=pd.get_dummies(df['color'])
df=df.join(dummy)
df.head()


,id,bone_length,rotting_flesh,hair_length,has_soul,color,type,bone_flesh,bone_hair,bone_soul,flesh_hair,flesh_soul,hair_soul,black,blood,blue,clear,green,white
0,0,0.354512,0.350839,0.465761,0.781142,clear,Ghoul,0.124377,0.165118,0.276924,0.163407,0.274055,0.363825,0,0,0,1,0,0
1,1,0.575560,0.425868,0.531401,0.439899,green,Goblin,0.245113,0.305853,0.253188,0.226307,0.187339,0.233763,0,0,0,0,1,0
2,2,0.467875,0.354330,0.811616,0.791225,black,Ghoul,0.165783,0.379735,0.370195,0.287580,0.280355,0.642171,1,0,0,0,0,0
3,4,0.776652,0.508723,0.636766,0.884464,black,Ghoul,0.395101,0.494546,0.686921,0.323937,0.449947,0.563196,1,0,0,0,0,0
4,5,0.566117,0.875862,0.418594,0.636438,green,Ghost,0.495840,0.236973,0.360298,0.366630,0.557432,0.266409,0,0,0,0,1,0


In [8]:
dummy_test=pd.get_dummies(df_test['color'])
df_test=df_test.join(dummy_test)
df_test.head()

,id,bone_length,rotting_flesh,hair_length,has_soul,color,bone_flesh,bone_hair,bone_soul,flesh_hair,flesh_soul,hair_soul,black,blood,blue,clear,green,white
0,3,0.471774,0.387937,0.706087,0.698537,black,0.183019,0.333114,0.329552,0.273917,0.270988,0.493228,1,0,0,0,0,0
1,6,0.427332,0.645024,0.565558,0.451462,white,0.275639,0.241681,0.192924,0.364798,0.291203,0.255328,0,0,0,0,0,1
2,9,0.549602,0.491931,0.660387,0.449809,black,0.270366,0.362950,0.247216,0.324865,0.221275,0.297048,1,0,0,0,0,0
3,10,0.638095,0.682867,0.471409,0.356924,white,0.435734,0.300804,0.227752,0.321910,0.243732,0.168257,0,0,0,0,0,1
4,13,0.361762,0.583997,0.377256,0.276364,black,0.211268,0.136477,0.099978,0.220317,0.161396,0.104260,1,0,0,0,0,0


In [9]:
cols=df.columns.tolist()
cols=cols[:6]+cols[7:]+cols[6:7]
df=df[cols]

In [10]:
le = preprocessing.LabelEncoder()
df['type']=le.fit_transform(df['type'].astype('str'))

df.head()

,id,bone_length,rotting_flesh,hair_length,has_soul,color,bone_flesh,bone_hair,bone_soul,flesh_hair,flesh_soul,hair_soul,black,blood,blue,clear,green,white,type
0,0,0.354512,0.350839,0.465761,0.781142,clear,0.124377,0.165118,0.276924,0.163407,0.274055,0.363825,0,0,0,1,0,0,1
1,1,0.575560,0.425868,0.531401,0.439899,green,0.245113,0.305853,0.253188,0.226307,0.187339,0.233763,0,0,0,0,1,0,2
2,2,0.467875,0.354330,0.811616,0.791225,black,0.165783,0.379735,0.370195,0.287580,0.280355,0.642171,1,0,0,0,0,0,1
3,4,0.776652,0.508723,0.636766,0.884464,black,0.395101,0.494546,0.686921,0.323937,0.449947,0.563196,1,0,0,0,0,0,1
4,5,0.566117,0.875862,0.418594,0.636438,green,0.495840,0.236973,0.360298,0.366630,0.557432,0.266409,0,0,0,0,1,0,0


In [11]:
x_train=df.drop(['color','id','type'],axis=1)
y_train=df['type']
x_test=df_test.drop(['color','id'],axis=1)

In [12]:
random_forest = RandomForestClassifier(n_estimators=100, min_samples_split=25, max_depth=7, max_features=1)
#random_forest = RandomForestClassifier(n_estimators=200)

random_forest.fit(x_train, y_train)

pred = random_forest.predict(x_test)

random_forest.score(x_train, y_train)

0.83557951482479786

In [13]:
clf = SVC(kernel='rbf',C=5,gamma=0.1)
clf.fit(x_train, y_train)
svm_pred=clf.predict(x_test)
clf.score(x_train, y_train)

0.76549865229110514

In [14]:
#pred=le.inverse_transform(pred)
svm_pred=le.inverse_transform(svm_pred)

In [15]:
submission = pd.DataFrame({
        "id": df_test["id"],
        "type": svm_pred
    })
submission.to_csv('Sample_Submission.csv',sep=',', index=False)